In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.staticfiles import StaticFiles
from starlette.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
import sys
from io import StringIO
import pandas as pd
import re

In [2]:
# boilerplate loading

load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Define request and response models
class QueryRequest(BaseModel):
    prompt: str
    headers: list
    sample: list # can be entire csv, remember to process
    
class QueryResponse(BaseModel):
    response: dict

In [15]:
# Helper funcs
def print_red(*strings):
  print('\033[91m' + ' '.join(strings) + '\033[0m')

# print msg in blue, , accept multiple strings like print statement
def print_blue(*strings):
  print('\033[94m' + ' '.join(strings) + '\033[0m')

def sanitize_input(query: str) -> str:
    """Sanitize input to the python REPL.
    Remove whitespace, backtick & python (if llm mistakes python console as terminal
    """

    # Removes `, whitespace & python from start
    query = re.sub(r"^(\s|`)*(?i:python)?\s*", "", query)
    # Removes whitespace & ` from end
    query = re.sub(r"(\s|`)*$", "", query)
    return query
    
def execute_panda_dataframe_code(code):
    """
    Execute the given python code and return the output. 
    References:
    1. https://github.com/langchain-ai/langchain-experimental/blob/main/libs/experimental/langchain_experimental/utilities/python.py
    2. https://github.com/langchain-ai/langchain-experimental/blob/main/libs/experimental/langchain_experimental/tools/python/tool.py
    """
     # Save the current standard output to restore later
    old_stdout = sys.stdout
    # Redirect standard output to a StringIO object to capture any output generated by the code execution
    sys.stdout = mystdout = StringIO()
    try:
		    # Execute the provided code within the current environment
        cleaned_command = sanitize_input(code)
        exec(code)
        
        # Restore the original standard output after code execution
        sys.stdout = old_stdout
				
				# Return any captured output from the executed code
        return mystdout.getvalue()
    except Exception as e:
        sys.stdout = old_stdout
        return repr(e)

def clean_spec(raw_text):
  cleaned_text = re.sub(r'^```json\n|\n```$|\\n|\\|^```|\n\s*|\r', '', raw_text)
  formatted_json = cleaned_text.strip()
  return formatted_json

In [292]:
tools = [
    {
        "type": "function",
        "function": {
            "strict": True,
            "name": "generate_chart",
            "description": """
            ALways try to call this function.
            Call this to generate a Vega-lite specification based on the user's inquiry for visualization. Requests can be short / terse and should relate to the data. Example user inquiries could be 'mpg over time' or 'cars by origins' if the csv is about cars.
            """,
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user's query. Can be re-written to be more comprehensible for Vega-lite generation, which will be handled by OpenAI.",
                    },
                    "context": {
                        "type": "string",
                        "description": "Relevant context that will be used in generating the Vega-Lite chart. Should include information such as headers, data sample, among any other facts that would be useful for chart generation.",
                    }
                },
                "required": ["query", "context"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "strict": True,
            "name": "analyze_data",
            "description": """Call this to generate Python code that uses pandas for data analysis. The code should assist in understanding metrics or performing tasks such as summarizing data, finding correlations, handling missing values, etc. 
            Example user inquiries could be:
            - 'summary statistics'
            - 'correlation between X and Y'
            - 'mean of X variable'. 
            
            Should return the desired metrics as a string.
            Should be ran multiple times in case you cannot generate all of the necessary metrics in one go or in the event that the code you ran failed.
            Very important is that you must begin with "global df", so you can have access to the varaible df, which is a pandas df of the user's provided data.
            
            You may also run this code in the event that you need to peek at some metrics in order to more fully reply to the user.
            """,
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The code to execute on a pandas df named 'df'. Must be written in Python and executable. Must end in a formatted print statement, e.g. print(f'[relevant info...]'), as its output will be captured by StringIO."
                    },
                },
                "required": ["code"],
                "additionalProperties": False,
            }}
    },
    {
        "type": "function",
        "function": {
            "strict": True,
            "name": "finalize_response",
            "description": """
            Call this after generating all of the needed information for an appropriate response. 
            If this tool is to be called, it should be the last tool called.

            You can draw from information provided in the chat history and from the tool results.
            
            In rare cases, if the user's inquiry is irrelevant or not related to data analysis, you may provide a blank Vega-lite spec with a description field that explains why the user's query is not being processed.
            """,
            "parameters": {
                "type": "object",
                "properties": {
                    "vega_spec": {
                        "type": "string",
                        "description": """The final Vega-lite spec (must be a JSON). Can be blank, but only when raising an error. 
                        Must have a descriptive 'description' field that talks about the data. If code was ran, it must summarize the metrics obtained.
                        If the user is being rejected, please inform the function of that as well."""
                    },
                },
                "required": ["vega_spec"],
                "additionalProperties": False,
        }}
    },
]


In [293]:
### Prompts
react_function_calling_prompt = '''You are a data analysis assistant using the ReAct style of reasoning for LLMs.

You run in a loop of Thought, Action, Observation in the following manner to answer the user question.
You MUST articulate your process in this structured manner:

Question: the input query you must answer in full
Thought: you should always think about what to do
Action: the action to take, should be one of tools provided
Action Input: the input to the action


You will be then call again with the result of the action.

Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
'''

vega_spec_prompt = ''' 
You are a data visualization assistant who will help the user with generating Vega-lite specifications.
Respond to the user's question faithfully and to the best of your ability. 

Your main task is to generate a Vega-lite spec (json), which is to aid in visualization of the data.

The user may ask for a scatterplot, bar chart, line chart, or any other visualization supported by Vega-lite. If the user does not specify, pick a visualization best compatible with the request and Vega-lite's limitations.
When making a graph in this way, try not to use the names of the entries unless specifically asked to. For example, the user may provide a csv of book names and ask you to list out the sales of a certain series. In that case, do not list out every book in the csv, but rather target that series specifically to the best of your ability.

You are to make very accurate predictions and realistic visualizations that are helpful and visually useful for the user.
Your output is strictly in JSON format only. You will be rewarded for doing an accurate job.
'''

synthesis_prompt = '''
Given the following chunk of text, you must now make a singular Vega-lite spec. It must be in JSON format. Your task is to merge the text outside of the JSON and alter the 'description' field of the final Vega-lite spec such that it has all of the relevant info to be communicated to the user. This description is less about the Vega-lite spec and more about communicating ideas about the data / answering the query to the user.

If the user is being rejected, please make sure to still provide a valid Vega-lite spec and make it such that the description is informing the user that they are being rejected because their query was not related.
'''

In [294]:
# Funcs
def create_vega_spec(query, context):
    messages = [
        {
            "role": "system",
            "content": vega_spec_prompt
        },
        {
            "role": "user",
            "content": f"Here is the context to aid you in Vega-lite spec generation: {context}"
        },
        {
            "role": "user",
            "content": f"This is my inquiry for which you are to generate a Vega-lite spec in JSON: {query}"
        },
    ]


    response = client.chat.completions.create( # Chat
            messages=messages,
            model="gpt-4o",
            response_format= { "type": "json_object" }
        )

    return response.choices[0].message.content

def create_analysis_code(code):
    sanitized = sanitize_input(code)
    result = execute_panda_dataframe_code(sanitized)


    return f"""
    ** BEGIN PYTHON CODE RESULT **
    {result}
    ** END PYTHON CODE REUSLT ** 
    
    """

def synthesize_final_ans(vega_spec):
    messages = [
        {
            "role": 'system',
            "content": synthesis_prompt
        },
        {
            "role": 'user',
            "content": str(vega_spec)
        }
    ]

    response = client.chat.completions.create( # Chat
            messages=messages,
            model="gpt-4o",
            response_format= { "type": "json_object" }
        )

    return response.choices[0].message.content

df = pd.DataFrame() # Store outside of funcs for global scope. Make sure to use "global" in relevant funcs.

tool_map = {
    'generate_chart': create_vega_spec,
    'analyze_data': create_analysis_code,
    'finalize_response': synthesize_final_ans
}

In [295]:
def query_openai(request: QueryRequest):
    try:
        global df
        df = pd.DataFrame((request.sample)) # Catch df
        to_json = None
        messages=[
            {
                "role": "system",
                "content": react_function_calling_prompt
            },
            {
                "role": "user",
                "content": f"Here are the headers: {', '.join(request.headers)}"
            },
            {
                "role": "user",
                "content": f"""Here is a subsection of the csv to better contextualize your answers: {df.head(20)} 
                """
            },
            {
                "role": "user",
                "content": f"""

                Your final Vega-Lite specification should at least have:
                - A "description" field that summarizes what the specification is.
                - The x-axis should be labeled.
                - The y-axis should be labeled.
                - Be in JSON format.
                - No URL field.

                Give me a Vega-Lite (JSON) specification that fulfills the following request: {request.prompt}"""
            },
        ]

        i = 0
        while i < 10: # Max of 10 iters, change if needed
            print(f"current iter:{i}")
            response = client.chat.completions.create( # Chat
                messages=messages,
                model="gpt-4o",
                tools=tools
            )

            if response.choices[0].message.content:
                print_red(response.choices[0].message.content) 
            
            messages.append(response.choices[0].message)

            if not response.choices[0].message.tool_calls:
                break

            for tool_call in response.choices[0].message.tool_calls:
                print_blue('calling:'+tool_call.function.name)           
                
                # call the function
                arguments = json.loads(tool_call.function.arguments)
                function_to_call = tool_map[tool_call.function.name]

                result = function_to_call(**arguments) # save outcome

                # create a message containing the tool call result
                result_content = json.dumps({
                    **arguments,
                    "result": result
                })
                function_call_result_message = {
                    "role": "tool",
                    "content": result_content,
                    "tool_call_id": tool_call.id
                }
                print_blue('action result:' + result_content)

                # save the action outcome for LLM
                messages.append(function_call_result_message)
            
                # break
                if tool_call.function.name == "finalize_response":
                    print("Hey, I'm done!")
                    to_json = result
                    i += 10
                    break

            i += 1

        try:
            response_json = json.loads(to_json)
            return QueryResponse(response=response_json)
        except json.JSONDecodeError:
            raise HTTPException(status_code=500, detail="Failed to decode JSON from OpenAI response. Apologies, but please try again.")

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [299]:
# dummy
# Create sample data for testing
test_headers = ["Name", "Money", "Items in Cart", "Sex"]
test_sample = [
    {"Name": "John", "Money": "123", "Items in Cart": "3", "Sex":"M"},
    {"Name": "Jane", "Money": "234", "Items in Cart": "5", "Sex":"F"},
    {"Name": "Alia", "Money": "733", "Items in Cart": "8", "Sex":"F"},
    {"Name": "Bobb", "Money": "345", "Items in Cart": "9", "Sex":"M"},
    {"Name": "Crom", "Money": "987", "Items in Cart": "3", "Sex":"M"},
]

# Create a QueryRequest object for testing
test_query_request = QueryRequest(
    prompt="avg money for each sex, tell me the mean money as well",
    headers=test_headers,
    sample=test_sample
)

end = query_openai(test_query_request)

current iter:0
calling:analyze_data
action result:{"code": "global df\nmean_money = df['Money'].mean()\nprint(f'Mean Money: {mean_money}')", "result": "\n    ** BEGIN PYTHON CODE RESULT **\n    TypeError(\"Could not convert string '123234733345987' to numeric\")\n    ** END PYTHON CODE REUSLT ** \n    \n    "}
calling:generate_chart
action result:{"query": "average money for each sex", "context": "Headers: Name, Money, Items in Cart, Sex; Data Sample: John, 123, 3, M; Jane, 234, 5, F; Alia, 733, 8, F; Bobb, 345, 9, M; Crom, 987, 3, M", "result": "\n{\n  \"$schema\": \"https://vega.github.io/schema/vega-lite/v5.json\",\n  \"description\": \"Average money for each sex.\",\n  \"data\": {\n    \"values\": [\n      {\"Name\": \"John\", \"Money\": 123, \"Items in Cart\": 3, \"Sex\": \"M\"},\n      {\"Name\": \"Jane\", \"Money\": 234, \"Items in Cart\": 5, \"Sex\": \"F\"},\n      {\"Name\": \"Alia\", \"Money\": 733, \"Items in Cart\": 8, \"Sex\": \"F\"},\n      {\"Name\": \"Bobb\", \"Money\":

In [301]:
end.response

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json',
 'description': 'This bar chart represents the average amount of money for each sex in the dataset. The calculated mean money value across all entries is 484.4, which provides insight into spending behavior as categorized by gender. Individuals are categorized as either male or female, showcasing the disparity, if any, between the two groups. It also considers the total number of items in the cart alongside the monetary value per transaction.',
 'data': {'values': [{'Name': 'John',
    'Money': 123,
    'Items in Cart': 3,
    'Sex': 'M'},
   {'Name': 'Jane', 'Money': 234, 'Items in Cart': 5, 'Sex': 'F'},
   {'Name': 'Alia', 'Money': 733, 'Items in Cart': 8, 'Sex': 'F'},
   {'Name': 'Bobb', 'Money': 345, 'Items in Cart': 9, 'Sex': 'M'},
   {'Name': 'Crom', 'Money': 987, 'Items in Cart': 3, 'Sex': 'M'}]},
 'mark': 'bar',
 'encoding': {'x': {'field': 'Sex',
   'type': 'nominal',
   'axis': {'title': 'Sex'}},
  'y': {'aggregat